# Vibrational Frequency Calculation with ASE and FAIRChem

This notebook demonstrates how to calculate vibrational frequencies and visualize vibrational modes of molecules using **ASE (Atomic Simulation Environment)** and the **FAIRChem machine-learning interatomic potential (MLIP)**.

As a case study, we will:
- Build a water molecule.
- Optimize its geometry using MLIP.
- Calculate vibrational frequencies.
- Visualize vibrational modes interactively.

## Further Reading

For more details, see [FAIRChem documentation](https://fair-chem.github.io/uma_tutorials/uma_tutorial.html#molecular-vibrations).


In [2]:
# Import essential packages from ASE
from ase import Atoms
from ase.optimize import BFGS, LBFGS
from ase.vibrations import Vibrations
from ase.build import molecule
from ase.io.trajectory import Trajectory
from ase.visualize import view

# Import FAIRChem for machine learning interatomic potentials
from fairchem.core import pretrained_mlip, FAIRChemCalculator
import fairchem.core

## Checking FAIRChem Version

It is always good practice to check the version of FAIRChem being used to ensure reproducibility.


In [3]:
fairchem.core.__version__

'2.3.0'

## Load Pretrained FAIRChem MLIP

We use the `uma-s-1` pretrained FAIRChem model as the interatomic potential.


In [4]:
# Load FAIRChem pretrained unit for MLIP prediction
predictor = pretrained_mlip.get_predict_unit("uma-s-1", device="cpu")

# Initialize the calculator
calc = FAIRChemCalculator(predictor, task_name="omol")

## Build and Optimize Water Molecule

We create a water molecule using ASE's built-in molecular builder.   
- The structure is optimized with the LBFGS optimizer 


In [5]:
# Create water molecule
water = molecule("H2O")

# Add spin and charge information
water.info.update({"spin": 1, "charge": 0})
water.calc = calc
LBFGS(water).run(fmax=0.001)

       Step     Time          Energy          fmax
LBFGS:    0 14:30:31    -2079.861719        0.483773
LBFGS:    1 14:30:31    -2079.864175        0.110125
LBFGS:    2 14:30:31    -2079.864371        0.026661
LBFGS:    3 14:30:32    -2079.864396        0.020670
LBFGS:    4 14:30:32    -2079.864427        0.004628
LBFGS:    5 14:30:32    -2079.864427        0.000617


np.True_

## Vibrational Analysis

We now compute vibrational frequencies using ASE's `Vibrations` class.  

- `indices=None` → All atoms are included in the vibrational analysis.  
- `delta=0.01` → Displacement step (in Å) used to compute force constants.  
- `name="vib-h2o"` → Name to use for output files and folders.  

Before running, we call `.clean()` to remove old vibration files if they exist.


In [6]:
vib = Vibrations(water, indices=None, delta=0.01, name="vib-h2o", nfree=2)
vib.clean()
vib.run()
vib.summary()

---------------------
  #    meV     cm^-1
---------------------
  0    2.0i     15.9i
  1    0.1i      0.6i
  2    0.0       0.2
  3    4.0      31.9
  4    4.2      33.9
  5   22.5     181.8
  6  200.9    1620.1
  7  473.7    3820.6
  8  485.6    3916.9
---------------------
Zero-point energy: 0.595 eV


## Write Vibrational Modes

For each vibrational mode, ASE can generate `.traj` files that store the atomic displacements.  
These can be visualized to see the atomic motions associated with each vibrational frequency.


In [7]:
# Loop through all vibrational modes and save trajectory files
for i in range(len(vib.get_energies())):
    vib.write_mode(i)

## Visualize Vibrational Modes

We can visualize vibrational modes interactively using ASE's visualization tools.  
Here, we use the **nglviewer** backend for interactive 3D visualization inside Jupyter.  

For example, let’s visualize mode 8:


In [9]:
mode = 8
traj = Trajectory(f"vib-h2o.{mode}.traj")
view(traj, viewer='ngl')

## Vibrational Frequency Calculation of Methanol (CH₃OH)

In this section, we perform vibrational analysis of methanol:

1. Build the CH₃OH molecule using ASE's molecular builder.  
2. Set spin and charge information.  
3. Optimize the geometry with the LBFGS algorithm (`fmax=0.001 eV/Å`).  
4. Run vibrational analysis with ASE's `Vibrations` class.  
5. Print a summary of the vibrational frequencies.  

This workflow follows the same procedure as for water, but now applied to methanol.


In [10]:
# for methanol
atoms = molecule("CH3OH")
atoms.info.update({"spin": 1, "charge": 0})
atoms.calc = calc
LBFGS(atoms).run(fmax=0.001)
vib = Vibrations(atoms, indices=None, delta=0.01, name="vib-ch3oh", nfree=2)
vib.clean()
vib.run()
vib.summary()

       Step     Time          Energy          fmax
LBFGS:    0 14:32:16    -3148.816399        0.563580
LBFGS:    1 14:32:16    -3148.819538        0.270434
LBFGS:    2 14:32:16    -3148.820411        0.068866
LBFGS:    3 14:32:16    -3148.820620        0.035621
LBFGS:    4 14:32:16    -3148.820684        0.028428
LBFGS:    5 14:32:16    -3148.820729        0.017354
LBFGS:    6 14:32:16    -3148.820750        0.010016
LBFGS:    7 14:32:16    -3148.820756        0.007310
LBFGS:    8 14:32:16    -3148.820759        0.006512
LBFGS:    9 14:32:17    -3148.820761        0.003647
LBFGS:   10 14:32:17    -3148.820762        0.002036
LBFGS:   11 14:32:17    -3148.820762        0.001896
LBFGS:   12 14:32:17    -3148.820763        0.001747
LBFGS:   13 14:32:17    -3148.820763        0.001335
LBFGS:   14 14:32:17    -3148.820762        0.000626
---------------------
  #    meV     cm^-1
---------------------
  0   12.4i    100.0i
  1    2.0i     16.1i
  2    0.0i      0.3i
  3    0.1       0.5
  